In [2]:
import pandas as pd

df = pd.read_csv("../raw_data/en.openfoodfacts.org.products.csv", delimiter='\t')

/tmp/ipykernel_928/988412624.py:3: DtypeWarning: Columns (0,8,13,19,20,21,22,23,27,28,29,31,32,40,41,42,52,55,64,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../raw_data/en.openfoodfacts.org.products.csv", delimiter='\t')


Nous supprimons certaines colonnes qui n'apporteront (a priori) rien pour le clustering:
- ```urlcreator, created_t, created_datetime, states, states_tags, last_modified_datetime, last_modified_t, image_url, image_small_url, image_nutrition_url, image_nutrition_small_url, states, states_en``` &rarr; Variables portant sur l'entrée du produit dans la bdd openfoodfacts
- ```product_name``` &rarr; A étudier lorsque la variable sera encodée pour voir les changements sur le modèle
- ```countries, countries_en``` &rarr; Redondant avec country_tags
- ```energy_100g``` &rarr; redondant avec ```energy-kcal_100g```
- ```brands``` &rarr; Nom de la marque.

In [3]:
columns = ['url', 'creator', 'created_t', 'created_datetime', 'states', 'states_tags', 'last_modified_datetime', 'last_modified_t',
'image_url', 'image_small_url', 'image_nutrition_url', 'image_nutrition_small_url', 'countries','countries_en',
'energy_100g', 'product_name', 'states', 'states_en', 'brands']

df.drop(columns=columns, inplace=True)
# to_drop = [col for col in columns if col in df.columns]
# df.drop(columns=to_drop, inplace=True)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059358 entries, 0 to 2059357
Columns: 169 entries, code to carnitine_100g
dtypes: float64(124), object(45)
memory usage: 2.6+ GB


Les colonnes restantes:

- ```pnns_groups_1``` et ```pnns_groups_2``` &rarr; Ces colonnes correspondent deja 
- ```fat_100g, saturated-fat_100g```, ```carbohydrates_100g```, ```sugars_100g```, ```proteins_100g```, ```salt_100g```, ```sodium_100g``` &rarr; Valeurs nutritionelles
- ```energy-kcal_100g``` &rarr; Les fabricants de produits alimentaires calculent cette valeur en additionnant où kcal/g de protéines, lipides et glucides contenus dans leur produit. [source](https://fr.wikipedia.org/wiki/Composition_nutritionnelle_des_aliments) &rarr; à vérifier si cette valeur n'est pas redondante avec les valeurs nutritionelles
- ```nutrition-score-fr_100g```
-  ```brands_tags``` &rarr; Nom de la marque sans caractères spéciaux. Un certain nombre de Nan mais une information qui est parfois utile lorsqu'elle est présente (Intuitivement par exemple nous savons qu'un produit de la marque Picard sera très probablement un produit surgelé). A encoder
- ```code``` &rarr; Cette variable correspond au code EAN13. Une partie du code correspond au pays et à l'entreprise emmetrice du produit [source](https://www.laintimes.com/explication-et-construction-des-codes-barres-suivant-la-norme-ean-13/)
A réencoder en 3 colonnes différentes, `Pays`, `Entreprise`, `Code Produit` et supprimer le dernier chiffre
- ```country_tags``` &rarr; Pays ou le produit est vendu

---